# Run in the beginning

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Illuin

/content/drive/MyDrive/Illuin


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch.nn as nn
import torch
import os
import json
from tqdm import tqdm
import pickle
from tqdm import tqdm

!pip install transformers
import transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.2 MB/s 
     |████████████████████████████████| 120 kB 71.2 MB/s 
     |████████████████████████████████| 6.6 MB 48.4 MB/s 


Data is organized this way:
<li> dic with 2 keys data and version
<li> data is an array of 442 dics
<li> each dic has 2 keys title and paragraphs (variable nb of paragraphs)
<li> each paragraph is a dic of context and qas
<li> qas is an array of dics of questions and answers and index of answer start is the context

# DataSet

In [281]:
with open('squad1.1/train-v1.1.json') as file:
    train_df = json.load(file)

with open('squad1.1/dev-v1.1.json') as file:
    dev_df = json.load(file)

In [295]:
data = train_df['data'].copy()
data2 = dev_df['data'].copy()

In [284]:
df = pd.DataFrame([])

for i in tqdm(range(len(data))):
    for j in range(len(data[i]['paragraphs'])):
        df = pd.concat((df, pd.DataFrame.from_dict(data[i]['paragraphs'][j])))

df = df.reset_index(drop=True).rename({'qas': 'questions'}, axis='columns')
df['questions'] = df['questions'].map(lambda x: x['question'])

100%|██████████| 442/442 [01:04<00:00,  6.84it/s]


In [288]:
df.head()

,context,questions
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...


In [ ]:
# pickling 

with open('pickled_df', 'rb') as file:
    pickle.dump(df, file)

# Basic IR

In [ ]:
def retrieve_context(question, context):

  #lem = WordNetLemmatizer()
  #stop_words = set(stopwords.words('english'))
  #question = " ".join([lem.lemmatize(word) for word in question.split() if word not in stop_words])
  context_set = context.copy().drop_duplicates().append(pd.Series(question)).reset_index(drop=True)
  #context_set_train = context_set.apply(lambda context: context.lower())
  #context_set_train = context_set.apply(lambda context: " ".join([lem.lemmatize(word) for word in context.split() if word not in stop_words]))
  
  
  tfidf = TfidfVectorizer()
  train_tfidf = tfidf.fit_transform(context_set)
  train_tfidf = list(train_tfidf.toarray())
  kdtree = KDTree(train_tfidf, leaf_size=3)
  distance, idx = kdtree.query([train_tfidf[len(train_tfidf)-1]], k=4)

  '''
  t = BertTokenizerFast.from_pretrained('bert-base-uncased')
  bert_tokenized = t(list(context_set), return_attention_mask=False, return_token_type_ids=False, truncation=True, padding=True)
  tokenized = np.array(bert_tokenized['input_ids'])
  tokenized = np.array([tokens/(10*sum(tokens>0)) for tokens in tokenized])
  kdtree = KDTree(tokenized, leaf_size=10)
  distance, idx = kdtree.query([tokenized[len(bert_tokenized['input_ids'])-1]], k=30)

  print(context_set[len(bert_tokenized['input_ids'])-1])
  print(len(context_set[idx[0][1]].split()))
  print(len(tokenized[0]))
  '''

  print(distance)
  print(idx)
  return context_set[idx[0][1]]


# IR with chunks

In [26]:
def create_dict_of_indices(chunk_size, list_contexts):
  lengths = [math.ceil(len(sentence.split())/chunk_size) for sentence in list_contexts]

  def cumulativeSum(arr):
    for i in range(1, len(arr)):
      arr[i] += arr[i - 1]
    return arr
  
  indices_before_range = [0] + cumulativeSum(lengths) # add 0 for range later (below)
  indices = [list(range(indices_before_range[i-1], indices_before_range[i])) for i in range(1, len(indices_before_range))]
  chunk_to_context = {indices[i][j]: i for i in range(len(indices)) for j in range(len(indices[i]))}

  return chunk_to_context

In [ ]:
def retrieve_context(question, context):
  chunks_length = 60
  context_set = list(context.copy().drop_duplicates().reset_index(drop=True))

  # to find the context later on from the chunks
  indice_chunk_to_context = create_dict_of_indices(chunks_length, context_set)

  #creating chunks of length chunks_length words
  context_set_split = [" ".join(context_set[j].split()[i:i+chunks_length]) for j in range(len(context_set)) for i in range(0, len(context_set[j].split()), chunks_length)]
  context_set_split.append(question)
  
  tfidf = TfidfVectorizer()
  train_tfidf = tfidf.fit_transform(context_set_split)
  train_tfidf = list(train_tfidf.toarray())
  kdtree = KDTree(train_tfidf, leaf_size=3)
  distance, idx = kdtree.query([train_tfidf[len(train_tfidf)-1]], k=4)

  print(question)
  print(idx)
  return context_set[indice_chunk_to_context[idx[0][1]]]


In [ ]:
count = 0
#for i in tqdm(range(1)):
for i in range(30):
  context = retrieve_context(df['questions'][i], df['context'][:5000])
  bval = context in list(df['context'][df['questions'] == df['questions'][i]])
  if bval:
    count+=1

print()
print(count)

# Chunks + capital

In [ ]:
def retrieve_context(question, context):
  chunks_length = 8
  context_set = context.copy().drop_duplicates().reset_index(drop=True)
  context_set_only_capitals = context_set.apply(lambda context: " ".join(re.findall(r'\b[A-Z][a-z]*[a-z]\b', context)))

  context_set_only_capitals = list(context_set_only_capitals)
  indice_chunk_to_context = create_dict_of_indices(chunks_length, context_set_only_capitals)

  context_set_split = [" ".join(context_set_only_capitals[j].split()[i:i+chunks_length]) for j in range(len(context_set_only_capitals)) for i in range(0, len(context_set_only_capitals[j].split()), chunks_length)]
  context_set_split.append(question)


  tfidf = TfidfVectorizer()
  train_tfidf = tfidf.fit_transform(context_set_split)
  train_tfidf = list(train_tfidf.toarray())
  kdtree = KDTree(train_tfidf, leaf_size=10)
  distance, idx = kdtree.query([train_tfidf[len(train_tfidf)-1]], k=4)

  #print(question)
  #print(context_set_only_capitals[idx[0][1]])
  print(idx)

#  return context_set[idx[0][1]]
  return context_set[indice_chunk_to_context[idx[0][1]]]


# Remove most occurences

In [ ]:
all_dic = {}
for dic in [dict(Counter(s.split())) for s in sentences]:
  for (key, value) in dic.items():
    if key not in all_dic.keys():
      all_dic[key]= value
    else:
      all_dic[key]+=value
max_occurences = sorted(all_dic.values(), reverse=True)[2]

words_to_remove = []
for key,val in all_dic.items():
  if val >= max_occurences:
    words_to_remove.append(key)

sentences = [(" ".join([word for word in cte.split() if word not in words_to_remove])) for cte in sentences]

# Hugging face model

In [ ]:
! pip install -U sentence-transformers

In [ ]:
def cosine(A, B):
  return np.dot(A,B)/(norm(A)*norm(B))

In [ ]:
def process_data(context):
  context_set = (context.copy().drop_duplicates().reset_index(drop=True)).tolist()
  context_set = [" ".join((re.findall(r'\b[A-Z][a-z]*[a-z]\b', context))) for context in context_set]
  SentenceTransformer('sentence-transformers/all-MiniLM-L12-v1')
  
  with open('processed_contexts.pickle', 'wb') as file:
    pickle.dump(context_set, file)

process_data(df['context'][:2000])

In [ ]:
def retrieve_context(question, path_processed_data, context_set):
  #chunks_length = 8
  with open(path_processed_data, 'rb') as file2:
    processed_contexts = pickle.load(file2)

  processed_contexts.append(question)
  model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v1')
  embeddings = model.encode(processed_contexts)

  kdtree = KDTree(embeddings, leaf_size=10)
  distance, idx = kdtree.query([embeddings[len(embeddings)-1]], k=3)

  print(idx)

  return context_set[idx[0][1]]

# Processing

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.neighbors import KDTree, KNeighborsClassifier
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from transformers import BertTokenizerFast
from collections import Counter
import math
import re
from numpy.linalg import norm
from sentence_transformers import SentenceTransformer
from collections import Counter

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

lem = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Process data

In [274]:
with open('pickled_df', 'rb') as file:
  df = pickle.load(file)

In [275]:
def process_data(context):
  chunks_length = 20
  context_set = (context.copy().drop_duplicates().reset_index(drop=True))#.tolist()

  context_set = context_set.apply(lambda s: s.lower())
  context_set = [(" ".join([lem.lemmatize(word) for word in cte.split() if word not in stop_words])) for cte in context_set]

  
  # dict linking context indices with chunk indices
  indice_chunk_to_context = create_dict_of_indices(chunks_length, context_set)

  # divide into chiunks
  context_set_split = [" ".join(context_set[j].split()[i:i+chunks_length]) for j in range(len(context_set)) for i in range(0, len(context_set[j].split()), chunks_length)]

  with open('processed_contexts.pickle', 'wb') as file:
    pickle.dump(context_set_split, file)

  with open('dict_indices.pickle', 'wb') as file:
    pickle.dump(indice_chunk_to_context, file)


In [276]:
process_data(df['context'][:2000])

## Perform prediction

In [277]:
def cosine(A, B):
  return np.dot(A,B)/(norm(A)*norm(B))


In [278]:
def retrieve_context(question, path_processed_data, path_chunks, context_set):

  context_set = (context_set.copy().drop_duplicates().reset_index(drop=True)).tolist()

  with open(path_processed_data, 'rb') as file:
    processed_contexts = pickle.load(file)

  with open(path_chunks, 'rb') as file:
      indice_chunk_to_context = pickle.load(file)

  question_processed = question.lower()
  question_processed = [" ".join(lem.lemmatize(word) for word in question_processed.split() if word not in stop_words)][0]
  processed_contexts.append(question_processed)

  tfidf = TfidfVectorizer()
  train_tfidf = tfidf.fit_transform(processed_contexts)
  train_tfidf = list(train_tfidf.toarray())

  kdtree = KDTree(train_tfidf, leaf_size=10)
  distance, idx = kdtree.query([train_tfidf[len(train_tfidf)-1]], k=4)



  actual_indices = np.array(list(indice_chunk_to_context.values()))[idx[0][1:]]
  corresponding_contexts = np.array(context_set)[actual_indices]
  corresponding_contexts = np.append(corresponding_contexts, question)

  model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
  embeddings = model.encode(corresponding_contexts)

  scores = []
  for i in range(len(embeddings)-1):
    scores.append(cosine(embeddings[i], embeddings[len(embeddings)-1]))


  print()
  print(actual_indices)
  print(scores)

  ind = np.argmax(scores)
  return corresponding_contexts[ind]


In [280]:
count = 0
for i in tqdm(range(1000)):
#for i in range(20):
  context = retrieve_context(df['questions'][i], './processed_contexts.pickle', './dict_indices.pickle', df['context'][:2000])
  bval = context in list(df['context'][df['questions'] == df['questions'][i]])
  if bval:
    count+=1
  
print()
print(count)


  0%|          | 1/1000 [00:02<33:56,  2.04s/it]


[ 0 28 38]
[0.6013752, 0.5003573, 0.47008035]


  0%|          | 2/1000 [00:03<29:49,  1.79s/it]


[ 0 42  7]
[0.5463146, 0.30903104, 0.53861564]


  0%|          | 3/1000 [00:05<30:42,  1.85s/it]

...


[118 260 254]
[0.6598302, 0.1156987, 0.043799926]


100%|█████████▉| 999/1000 [30:36<00:01,  1.64s/it]


[118 118 176]
[0.5493145, 0.5493145, 0.06564085]


100%|██████████| 1000/1000 [30:37<00:00,  1.84s/it]


[ 66  27 118]
[0.38717645, -0.022682838, 0.4483493]

637
